# Global Sequence Alignment with Needleman-Wunsch

This notebook shows the how a list of partial transcripts are aligned with a full transcript using the _Needleman-Wunsch_ algorithm.

## How it works

The _Needleman Wunsch_ (NW) algorithm alignms string $B$ with string $A$ by deleting characters or inserting gaps in one of the strings. It is very similar to the _Smith Waterman_ (SW) algorithm. However, whereas the latter performs a local aligment, NW performs a global alignment, i.e. the optimal alignment for all partial transcripts is found. Like the SW algorithm, NW calculates an alignment matrix first, containing the costs for insertion/deletion of characters and then backtracks that matrix.

## The extended algorithm

To align the partial transcript with the full transcript, they are concatenated and separated by space. The concatenated transcripts are used as string $B$ (the string to be aligned) and the full transcript as string $A$ (the reference string). The implementation for NW used in this project was inspired by [this implementation by Aleksander Levchuck](https://github.com/alevchuk/pairwise-alignment-in-python/blob/master/alignment.py). However some optimizations and additions were made.

Because we do not want to lose the information where the start and endings of a partial transcript are aligned, the _boundaries_, i.e. the indices of the positions in $B$ where a partial transcript starts and ends must be supplied to the function. The following example sentence serves as an example (the partial alignments are single words).

In [ ]:
from util.gsa_util import needle_wunsch

a = 'ich bin ein berliner'
partial_transcripts = ['ach', 'bene', 'ain', 'berlinör']
b = ' '.join(partial_transcripts)
boundaries = []
for p in partial_transcripts:
    boundaries += [(b.index(p), b.index(p) + len(p))]
    
print('A:', a)
print('B:', b)
print('boundaries:', boundaries)
print()

alignments, source_str, target_str = needle_wunsch(a, b, boundaries)
print('A (aligned):', source_str)
print('B (aligned):', target_str)
print()
for t, al in zip(partial_transcripts, alignments):
    start = al['start']
    end = al['end']
    text = al['text']
    
    print(f'partial transcript: {t}, aligned text: {text} (A[{start}:{end}])')

## Calculating the boundaries

Calculating the boundaries is cumbersome. That's why it is integrated in the LSA-Stage of the pipeline.

In [ ]:
from pipeline import gsa

transcript = 'ich bin ein berliner'
partial_transcripts = ['ach', 'bene', 'ain', 'berlinör']

alignments = gsa(transcript, partial_transcripts, align_endings=True)
    
for t, al in zip(partial_transcripts, alignments):
    start = al['start']
    end = al['end']
    text = al['text']
    
    print(f'partial transcript: {t}, aligned text: {text} (A[{start}:{end}])')

GSA also works for longer texts, although it can become quite slow then.

In [ ]:
from util.string_util import normalize
from pipeline import gsa

transcript = """
My fellow Americans,

This week, prosecutors, law enforcement officers, homeland security officials, and lawmakers joined me at the White House to address a very vicious threat to our communities—the savage gang, MS-13.

Glaring loopholes in our laws have allowed criminals and gang members to break into our country. For example, under current law, unaccompanied alien minors at the border are released into American communities no matter where, no matter how, it is so easy for them because the laws are bad and they have to be changed. This loophole is easily exploited by MS-13, which now operates in at least 40 states.

In addition to MS-13, many other gangs are breaking into our country routinely because our laws are so weak, so sad, so pathetic.

At this week’s roundtable, we learned the story of one family right here in Washington, D.C. that hosted a person in their home, who then began to recruit their young son to MS-13. When the boy’s mother tried to stop it, the gang member shot her in the head, blinding her for life. She’s lucky she lived, but she’s paying a very big price.

During my State of the Union, I called on Congress to immediately close dangerous loopholes in federal law that have endangered our communities and imposed enormous burdens on U.S. taxpayers.

My Administration has identified three major priorities for creating a safe, modern and lawful immigration system: fully securing the border, ending chain migration, and canceling the visa lottery. Chain migration is a disaster and very unfair to our country. The visa lottery is something that should have never been allowed in the first place. People enter a lottery to come into our country. What kind of a system is that? It is time for Congress to act and to protect Americans. Every member of Congress should choose the side of law enforcement, and the side of the American People. That’s the way it has to be.
"""
print(f'transcript (A) length: {len(transcript)}')

partial_transcripts = """
my fellow Americans
this week prosecutors law enforcement offices Homeland Security officials
lawmakers join me at the White House to address
a very vicious threat to our communities
The Savage gang MS-13
glaring loopholes in our laws have allowed criminals and gang members to break into our country
for example
under current law unaccompanied alien minors at the border
are released into American communities no matter where
no matter how
so easy for them because the laws are bad and they have to be changed
this loophole is easily exploited by MS-13
which now operates in at least 40 States
in addition to MS-13
many other gangs are breaking into our country routinely
because I lost the show week
so sad
at this week's Roundtable we learned the story of one family right here in Washington DC
that hosted a person in their home
who then began to recruit their young son
do MS-13
when the boy's mother tried to stop it the gang member shot her in the head
blinding her for life
she's lucky she lived but she's paying a very big price
during my state of the union I called on Congress to immediately close dangerous loopholes in federal law
that have endangered our communities
an imposed enormous burdens on us taxpayers
my Administration has identified three major priorities
creating a safe
modern and lawful immigration system fully security the Border
ending chain migration
I'm cancelling the Visa Lottery
chain migration is a disaster and very unfair to our country
the Visa Lottery is something that should have never been allowed in the first place
people enter a lottery to come into our country
what kind of a system is that
is time for Congress to act and to protect Americans
every member of Congress should choose the side of law enforcement
by the side of the American people
that's the way it has to be
"""
partial_transcripts = [line.strip() for line in partial_transcripts.splitlines() if line.strip()]
print(f'{len(partial_transcripts)} partial transcripts')
b = ' '.join(partial_transcripts)
print(f'concatenated partial transcripts (B) length: {len(b)}')

alignments = gsa(transcript, partial_transcripts, align_endings=True)

for p, al in zip(partial_transcripts, alignments):
    print('partial transcript:', p)
    print('aligned text      :', al['text'])
    print()

## Aligning the partial transcripts

The _Precision_ of alignments is calculated as the _Levenshtein Similarity_ between a partial transcript and its aligned text. _Recall_ is measured as how much of the full transcript is covered by the aligned texts. Because alignments should neither start nor end within words, the start and/or end index of an alignment is moved so it falls on a beginning or end of a word (whichever is closer). If the start/end index of an alignment happens to fall within a sequence of whitespace characters (space, newline, ...) it is moved to the right (start index) resp. to the left (end index) until a non-whitespace character is encountered. This prevents aligned texts containing spaces or line breaks at the start and beginning.

### Handling boundaries

With given boundaries the start of each partial transcript will be aligned in the full transcript. Aligning the end of each transcript (method 1) is optional. Doing so will lead to more accurate alignemnts (higher Precision) because each aligned text will have been aligned exactly with the characters of the corresponding partial transcript. On the other hand, aligning both start and end of each partial transcript will possibly lead to unaligned characters in the transcript (lower Recall). Consider the following example:

```
Full transcript    : I see, I see, said the blind man to his deaf daughter.
   alignments      : xxxxx---------xxxxxxxxxxxxxx-----xxxxxxxxxxxxxxxxxxxx-
Partial transcripts: i see         sed da blnd        dois death dooter
   Precision          1.0              0.57                 0.55

xxx = aligned parts
--- = unaligned parts

av. Precision: 0.71
av. Recall: 0.65
```

When not aligning the ends of the alignments (method 2) will simply move the end of each alignment to where the next alignment starts. This will prevent having unaligned text in the transcript (high Recall), although the alignments may contain text that is not spoken or would actually belong to the next alignment. Note that unaligned text is still possible at the very start or end of the full transcript. Consider the same example as above, but this time not aligning the endings:

```
Full transcript    : I see, I see, said the blind man to his deaf daughter.
   alignments      : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Partial transcripts: i see         sed  da  blnd      dois death dooter
   Precision             0.36          0.37                 0.52

xxx = aligned parts
--- = unaligned parts

av. Precision: 0.42
av. Recall: 1.00
```

Above procedures show the trade-off which usually happens when dealing with Precision and Recall: You can't have high values for both. Because a lower Recall is not necessarily bad (there may be annotations, footnotes, etc., which are not read), the default alignment method is to align both start and end of each partial transcript.